Load the hands, rocks and papers images, in total 2749.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import random
import os
import sys
import time
import torch.utils.data as Data
from PIL import Image
from sklearn.model_selection import train_test_split
#data loader, load all the png images in the path loader and save them in a list and return the list as a numpy
def data_loader(path,label):
    data_list = []
    label_list = []
    #search the path and load all the png images
    for file in os.listdir(path):
        if file.endswith(".png"):
            img = Image.open(path + "/"+ file)
            #resize the image to 32*32
            img = img.resize((128,128))
            img = np.array(img)
            data_list.append(img.reshape(128,128,3))
            label_list.append(label)
    #transform the list into a numpy array
    data_list = np.array(data_list)
    label_list = np.array(label_list)
    
    #reshape the numpy array into 2 dimensions
    # data_list = data_list.reshape(data_list.shape[0],1,data_list.shape[1],data_list.shape[2]) 
    return data_list,label_list
data_paper,paper_label=data_loader('../../db/paper',0)
data_rock,rock_label = data_loader('../../db/rock',1)
data_scissor,scissor_label = data_loader('../../db/scissors',2)
data_list = np.concatenate((data_paper,data_rock,data_scissor))
label_list = np.concatenate((paper_label,rock_label,scissor_label))



Split the data into training and testing parts

In [2]:
import tensorflow as tf
import numpy as np 
from tensorflow.keras.datasets import mnist

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
import tensorflow.keras.utils
print(data_list.shape)
#standardize the data
data = data_list
label=label_list
#split the data into train and test
X_train,X_test,Y_train,Y_test = train_test_split(data,label,test_size=0.2,random_state=42)
# change to one hot encoding
Y_train = tensorflow.keras.utils.to_categorical(Y_train, 3)
Y_test = tensorflow.keras.utils.to_categorical(Y_test, 3)
print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)


(2749, 128, 128, 3)
(2199, 128, 128, 3) (2199, 3) (550, 128, 128, 3) (550, 3)


Define 3 layers shallow convolution networks, use ImageDataGenerator to do the data augmentation

In [3]:


#imagedatagenetator

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=True,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=True,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=60,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

# create model
model = Sequential()

model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same', input_shape=(128,128,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))   # 
model.add(Dropout(0.2))                     # 

model.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])



In [5]:
#print the model using plot_model
import pydot

from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model.png')



('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [ ]:
datagen.fit(X_train)
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = "cnn.h5"
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)
# checkpoint
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.fit(datagen.flow(X_train,Y_train),
          validation_data=(X_test,Y_test),
          epochs=100,
          callbacks=[model_checkpoint_callback])

In [5]:
#load the model by using tensorflow
from keras.models import load_model
test_model = load_model('../saved_models/cnn.h5')
print(test_model.evaluate(X_test, Y_test))

18/18 [==============================] - 4s 190ms/step - loss: 22.8302 - accuracy: 0.9455
[22.830196380615234, 0.9454545378684998]


In [22]:
#paper 0 ,rock 1, scissor 2
import matplotlib.pyplot as plt
img = Image.open("/Users/xiewenxuan/Downloads/rock-paper-scissors copy/db/test.png")
#resize the image to 32*32
img = img.resize((32,32))
img = np.array(img)
img = img.reshape(1,32,32,3)
print(img.shape)


print(test_model.predict(img))

(1, 32, 32, 3)
[[0. 1. 0.]]
